In [28]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [29]:
original_df = pd.read_csv("GlobalLandTemperaturesByCountry+GHG.csv")
original_df.head()

,Unnamed: 0,year,Country,continent,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff,Code,gdp
0,0,1900,AFGHANISTAN,Asia,13.749333,0.748833,33.00,65.0,0.0,0.0,NaN,NaN
1,1,1900,ALBANIA,Europe,13.068583,0.723417,41.00,20.0,0.0,0.0,NaN,NaN
2,2,1900,ALGERIA,Africa,22.864167,0.826667,28.00,3.0,0.0,0.0,NaN,NaN
3,3,1900,AMERICAN SAMOA,Oceania,26.273500,0.891000,-14.33,-170.0,NaN,0.0,NaN,NaN
4,4,1900,ANDORRA,Europe,11.348333,0.542167,42.50,1.5,0.0,0.0,NaN,NaN


In [30]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24342 entries, 0 to 24341
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     24342 non-null  int64  
 1   year                           24342 non-null  int64  
 2   Country                        24342 non-null  object 
 3   continent                      24342 non-null  object 
 4   AverageTemperature             24341 non-null  float64
 5   AverageTemperatureUncertainty  24341 non-null  float64
 6   Latitude                       24342 non-null  float64
 7   Longitude                      24342 non-null  float64
 8   GHG                            22458 non-null  float64
 9   temp_diff                      24341 non-null  float64
 10  Code                           9396 non-null   object 
 11  gdp                            7323 non-null   float64
dtypes: float64(7), int64(2), object(3)
memory usag

In [31]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country","continent"]).sum().reset_index()
    
    return df

In [32]:
df = get_temperature_diff_between(original_df,1900,2013)
df.head()

,Country,continent,Unnamed: 0,year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff,gdp
0,AFGHANISTAN,Asia,1373406,223041,1622.516542,56.453375,3762.00,7410.0,2.618667e+09,2.784292,1.740719e+11
1,ALBANIA,Europe,1373520,223041,1469.704458,49.245000,4674.00,2280.0,7.362570e+09,1.925292,1.597039e+11
2,ALGERIA,Africa,1373634,223041,2652.966583,58.971792,3192.00,342.0,6.710646e+10,2.257333,2.965330e+12
3,AMERICAN SAMOA,Oceania,1373748,223041,3046.290000,55.131417,-1633.62,-19380.0,0.000000e+00,1.243750,6.712000e+09
4,ANDORRA,Europe,1373862,223041,1306.972958,42.170000,4845.00,171.0,1.406610e+08,0.959542,6.005083e+10


In [33]:
df["gdp_test"] = df["gdp"]*2

In [35]:
fig = px.scatter(df, x="temp_diff", y="GHG", color="continent", template="simple_white", hover_name="Country",
                 size = "gdp_test",size_max=80,
                labels={
                     "temp_diff": "Increase in average temperature",
                     "GHG": "Greenhouse Gases Emissions"

                 },)
fig.show()

In [30]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country"]).sum()[["temp_diff"]]
    
    return df